# TABLE OF CONTENT

xx
xx
xx


# PROBLEM DESCRIPTION

xx
xx
xx

***

# THE DATA


In [6]:
import pandas as pd
df = pd.read_csv('credits.csv')

In [33]:
df.iloc[1:10]

print(f"Total:{len(df)}")
print(f"First chunk: {0} - {len(df)/4}")
print(f"Second chunk: {(len(df)/4) +1} - {((len(df)/4)*2)}")
print(f"Third chunk: {((len(df)/4)*2) +1} - {((len(df)/4)*3)}")
print(f"Four chunk: {((len(df)/4)*3) +1} - {((len(df)/4)*4)}")

Total:45476
First chunk: 0 - 11369.0
Second chunk: 11370.0 - 22738.0
Third chunk: 22739.0 - 34107.0
Third chunk: 34108.0 - 45476.0


In [35]:
df[0:int(len(df)/4)].to_csv('credits01.csv')

In [ ]:
df[int((len(df)/4) +1):int(((len(df)/4)*2))].to_csv('credits02.csv')

In [ ]:
df[int(((len(df)/4)*2) +1):int(((len(df)/4)*3))].to_csv('credits03.csv')

In [ ]:
df[int(((len(df)/4)*3) +1):int(((len(df)/4)*4))].to_csv('credits04.csv')